In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from lsst.sims.photUtils import Sed
from lsst.sims.photUtils import Bandpass
from lsst.sims.photUtils import PhotometricParameters

import os

In [2]:
# Let's find the full well magnitude per filter for a variety of exposure times.
filters = ['u', 'g', 'r', 'i', 'z', 'y']



In [3]:
band_dir = os.getenv("LSST_THROUGHPUTS_BASELINE")
bps = []

for filtername in filters:
    
    tempB = Bandpass()
    tempB.readThroughput(band_dir+'total_'+filtername+'.dat')
    bps.append(tempB)

In [4]:
bps

In [5]:
wave = np.arange(bps[0].wavelen.min(), bps[-1].wavelen.max()+1)

In [6]:
const = 1e-3
flat_sed = Sed(wavelen=wave, fnu=np.zeros(wave.size, dtype=float) + const)

In [7]:
flat_sed.calcMag(bps[3])

16.400065622282231

In [8]:
params = PhotometricParameters(exptime=15, nexp=1)

In [9]:
flat_sed.calcADU(bps[0], params) 

65474.60911999298

In [10]:
params.gain  # Electrons per ADU

2.3

In [36]:
# Do I need to correct for this being a 2-d Gaussian?
def electrons_in_peak(adu, fwhm_eff, phot_params):
    # Given total number of ADUs from a star, and seeing,
    # return the 
    c = fwhm_eff/2.3548/phot_params.platescale
    peak = adu/(c**2*2.*np.pi)
    peak = peak*phot_params.gain
    return peak

In [37]:
# OK, let's make arrays of mags and peak electron counts.
electrons_in_peak(flat_sed.calcADU(bps[4], params) , 1., params)


104581.11683615333

In [38]:
# would like to know how we lose things on the bright end.  Let's start with r

In [39]:
consts = 10.**np.arange(-5, -2, .1)

In [50]:

fwhm = 1.
elec_max = 90e3
bandpass = bps[1]
mag_peaks = []
exptimes = [1, 15, 30, 60]
for exptime in exptimes:
    params = PhotometricParameters(exptime=exptime, nexp=1)
    mags = []
    peak_es = []
    for con in consts:
        flat_sed = Sed(wavelen=wave, fnu=np.zeros(wave.size, dtype=float) + con)
        peak_es.append(electrons_in_peak(flat_sed.calcADU(bandpass, params), fwhm, params))
        mags.append(flat_sed.calcMag(bandpass))

    mag_peak = np.interp(np.log10(elec_max), np.log10(peak_es), mags)
    mag_peaks.append(mag_peak)

In [51]:
mag_peaks

[14.150065622282256,
 15.186738671948364,
 15.939313661108317,
 16.691888650268268]